# Fit models to human data

In [ ]:
import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from Config.config import PATHS
from Classes.parameter_recovery import ParameterFit
from Classes.cognitive_model_agents import MODELS

### Load human data

In [ ]:
# Load data into a dataframe

two_player = False  # Set to True for 2-player UR experiment, False for multi-player IU experiment
if two_player:
	file_name = '2-player-UR.csv' # <= Data from 2-player UR experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_2P.json')
else:
	file_name = 'multi-player.csv' # <= Data from multi-player IU experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_MP.json')

data_folder = PATHS['human_data']
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')

data = pd.read_csv(file)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head(2)

### Select models

In [ ]:
# check_out_these = [model.name() for model in MODELS]
# check_out_these = ['Payoff-M2', 'Fairness-M2', 'AvailableSpace-M2']
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Payoff-M2', 'Fairness-M2']
# check_out_these = ['Fairness-M2']
check_out_these = ['WSLS-M1']
# check_out_these = ['FRA']
# check_out_these = ['FRA+Payoff+Attendance']
my_models = [model for model in MODELS if model.name() in check_out_these]

### Fit with scipy

In [ ]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Scipy.json',
    optimizer_type='scipy',
    hyperparameters=None,
    new_file=True
)

### Fit with Bayesian Optimizer

In [ ]:
hyperparameters = {
    'init_points':128,
    'n_iter':64
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json',
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=True
)

### Keep best fit from both optimizers

In [ ]:
df_2P_Scipy = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Scipy.json', lines=True)
df_2P_Bayesian = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json', lines=True)

df = pd.merge(left=df_2P_Scipy, right=df_2P_Bayesian, on='model_name', how='outer')

df_list = []
for model in my_models:
    df1 = df[df['model_name'] == model.name()]
    deviance_x = -df1['deviance_x'].unique()[0]
    deviance_y = -df1['deviance_y'].unique()[0]
    if deviance_x < deviance_y:
        df1.drop(columns=[col for col in df1.columns if '_y' in col], inplace=True)
        df1.rename(columns={col: col.replace('_x', '') for col in df1.columns}, inplace=True)
    else:
        df1.drop(columns=[col for col in df1.columns if '_x' in col], inplace=True)
        df1.rename(columns={col: col.replace('_y', '') for col in df1.columns}, inplace=True)
    df_list.append(df1)
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df

### Save on file

In [ ]:
new_file = False

if new_file:
    df.to_json(best_fits_file, orient='records', lines=True)
else:
    list_fixed_parameters_new = df['fixed_parameters'].values
    best_fit = pd.read_json(best_fits_file, lines=True)
    for fixed_parameters_new in list_fixed_parameters_new:
        df1 = df[df['fixed_parameters'] == fixed_parameters_new]
        for model in my_models:
            df2 = df1[df1['model_name'] == model.name()]
            if model.name() not in best_fit['model_name'].values:
                best_fit = pd.concat([best_fit, df1], ignore_index=True)
            else:
                best_fit.loc[best_fit['model_name'] == model.name(), df2.columns] = df2.values

    best_fit.to_json(best_fits_file, orient='records', lines=True)

---